<a href="https://colab.research.google.com/github/lucianeb/colab/blob/main/01_alocacao_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importar libraries
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Listar os ativos
tickers = ['PETR4.SA', 'ITUB4.SA', 'LREN3.SA', 'VALE3.SA', 'EMBR3.SA']

# Download de dados históricos
data = yf.download(tickers, start="2020-01-01", end="2024-07-31")['Adj Close']

# Cálculo dos retorno
returns = data.pct_change().dropna()

# Definição do portfolio de pesos iguais
def equal_weighted_portfolio(n_assets):
    return np.ones(n_assets) / n_assets

# Definição do portfolio de paridade de riscos
def risk_parity_portfolio(cov_matrix):
    inv_vol = 1 / np.sqrt(np.diag(cov_matrix))
    weights = inv_vol / np.sum(inv_vol)
    return weights

# Definição do portfolio de mínima variância
def min_variance_portfolio(cov_matrix):
    n_assets = cov_matrix.shape[0]
    args = (cov_matrix,)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(n_assets))
    result = minimize(portfolio_variance, n_assets * [1. / n_assets,], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# Definição do portfolio de Markovitz
def markowitz_portfolio(returns, cov_matrix, risk_free_rate=0.0):
    n_assets = cov_matrix.shape[0]
    mean_returns = returns.mean() * 252  # Annualize returns
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(n_assets))
    result = minimize(neg_sharpe_ratio, n_assets * [1. / n_assets,], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    portfolio_return = np.sum(mean_returns * weights)
    portfolio_volatility = np.sqrt(weights.T @ cov_matrix @ weights)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    return -sharpe_ratio

# Calcular os pesos de cada portfolio

# número de ativos
n_assets = returns.shape[1]

# matriz de covariância
cov_matrix = returns.cov() * 252  # matriz de covariância anualizada

# Calculo dos pesos para cada modelo
equal_weights = equal_weighted_portfolio(n_assets)
risk_parity_weights = risk_parity_portfolio(cov_matrix)
min_variance_weights = min_variance_portfolio(cov_matrix)
markowitz_weights = markowitz_portfolio(returns, cov_matrix)

# Tabela com os pesos de cada modelo

# construir dataframe
portfolio_weights = pd.DataFrame({
    'Equal-Weighted': equal_weights,
    'Risk Parity': risk_parity_weights,
    'Minimum Variance': min_variance_weights,
    'Markowitz': markowitz_weights
}, index=tickers)

# Formatar tabela de resultados
portfolio_weights = portfolio_weights * 100
portfolio_weights = portfolio_weights.applymap(lambda x: f"{x:.1f}%")

# Mostrar o resultado
print(portfolio_weights)